In [1]:
import sys
sys.path.append('../src')

In [2]:
from analysis_utils import SampleAggregator
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
aggregator = SampleAggregator()

In [4]:
df = aggregator.aggregate("deepseek-ai_deepseek-coder-1.3b-instruct")

Postprocessing Data: 100%|██████████| 11061/11061 [00:00<00:00, 118053.35it/s]


In [ ]:
df.head()

### Visualize Pass@1 By Mutation

In [ ]:
# Make a heatmap that plots the df['mutation'] column against pass@1 column, with the mean of the pass@1 column as the value
mean_pass = df.groupby('mutation')['pass_at_1_log_ratio'].median().reset_index()

In [17]:
mean_pass

,mutation,pass_at_1_log_ratio
0,AddParensTransformer,0.000000
1,AdditionInversionTransformer,0.000000
2,ArrayToDictTransformer,-2.033087
3,BlockCommentsTransformer,0.000000
4,ConstantSplittingTransformer,-0.017392
5,DictInitializerUnpackTransformer,-1.876279
6,DictToArrayTransformer,-2.237345
7,EmptyArrayToStringTransformer,-3.365566
8,EnumerateForTransformer,-0.047567
9,ExpandAugmentedAssignTransformer,0.000000


In [5]:
def minPath(grid, k):
    """
    Given a grid with N rows and N columns (N >= 2) and a positive integer k,
    each cell of the grid contains a value. Every integer in the range [1, N * N]
    inclusive appears exactly once on the cells of the grid.
    You have to find the minimum path of length k in the grid. You can start
    from any cell, and in each step you can move to any of the neighbor cells,
    in other words, you can go to cells which share an edge with you current
    cell.
    Please note that a path of length k means visiting exactly k cells (not
    necessarily distinct).
    You CANNOT go off the grid.
    A path A (of length k) is considered less than a path B (of length k) if
    after making the ordered lists of the values on the cells that A and B go
    through (let's call them lst_A and lst_B), lst_A is lexicographically less
    than lst_B, in other words, there exist an integer index i (1 <= i <= k)
    such that lst_A[i] < lst_B[i] and for any j (1 <= j < i) we have
    lst_A[j] = lst_B[j].
    It is guaranteed that the answer is unique.
    Return an ordered list of the values on the cells that the minimum path go through.
    Examples:
        Input: grid = [ [1,2,3], [4,5,6], [7,8,9]], k = 3
        Output: [1, 2, 1]
        Input: grid = [ [5,9,3], [4,1,6], [7,8,2]], k = 1
        Output: [1]
    """
    m, n = len(grid), len(grid[0])
    visited = [[-1] * n for i in range(m)]
    if k >= m * n:
        return grid[0][0]

    def dfs(row, col):
        if row < 0 or row == m or col < 0 or col == n:
            return 1
        if visited[row][col] == -1:
            visited[row][col] = grid[row][col] + min(
                dfs(row - 1, col), dfs(row, col - 1), dfs(row + 1, col), dfs(row, col + 1)
            )
        return visited[row][col]

    return dfs(0, 0)

In [6]:
minPath([[1,2,3], [4,5,6], [7,8,9]], 3)

RecursionError: maximum recursion depth exceeded